# Deutsch-Jozsa Algorithm
This algorithm queries an oracle of n-bit boolean inputs to check wheter it is constant or balanced. It does it with just one query, while classically n queries are needed. So, there is a significant computational speedup, from a time complexity of O(n) to O(1).

In [1]:
# import the Cirq library
import cirq

## Deutsch's algorithm on two qubits in Cirq

In [2]:
# Get two qubits, a data qubit and target qubit, respectively
q0, q1 = cirq.LineQubit.range(2)

We define a dictionary which maps oracle names to the operations in the circuit need to enanct the relative function

In [3]:
# Dictionary of oracles
oracles = {'0':[], '1':[cirq.X(q1)], 'x':[cirq.CNOT(q0,q1)], 'notx':[cirq.CNOT(q0,q1),cirq.X(q1)]}

In [4]:
def deutsch_algorithm(oracle):
    """Yields a circuit for Deutsch's algorithm given operations implementing the oracle"""
    yield cirq.X(q1)
    yield cirq.H(q0), cirq.H(q1)
    yield oracle
    yield cirq.H(q0)
    yield cirq.measure(q0)

In [5]:
# Display each circuit for all oracles
for key, oracle in oracles.items():
    print('Circuit for {}...'.format(key))
    print(cirq.Circuit(deutsch_algorithm(oracle)),end="\n\n")

Circuit for 0...
0: ───H───H───M───

1: ───X───H───────

Circuit for 1...
0: ───H───H───M───

1: ───X───H───X───

Circuit for x...
0: ───H───────@───H───M───
              │
1: ───X───H───X───────────

Circuit for notx...
0: ───H───────@───H───M───
              │
1: ───X───H───X───X───────



In [6]:
# Get a simulator
simulator = cirq.Simulator()

# Execute the circuit for each oracle to distinguish constant from balanced
for key, oracle in oracles.items():
    result = simulator.run(
        cirq.Circuit(deutsch_algorithm(oracle)),
        repetitions=10
    )
    print('oracle: {:<4} results: {}'.format(key, result))

oracle: 0    results: q(0)=0000000000
oracle: 1    results: q(0)=0000000000
oracle: x    results: q(0)=1111111111
oracle: notx results: q(0)=1111111111


### Deutsch-Jozsa's algorithm on three qubits

In [1]:
import cirq

# Get three qubits, two data qubits and one target qubit, respectively
q0, q1, q2 = cirq.LineQubit.range(3)

In [9]:
# Oracles for constant functions
constant = ([], [cirq.X(q2)])

# Oracles for balnced functions
balanced = ([cirq.CNOT(q0,q2)], [cirq.CNOT(q1,q2)],
            [cirq.CNOT(q0,q2)], [cirq.CNOT(q1,q2)],
            [cirq.CNOT(q0,q2)], [cirq.X(q2)],
            [cirq.CNOT(q1,q2)], [cirq.X(q2)],
            [cirq.CNOT(q0,q2)], [cirq.CNOT(q1,q2)],[cirq.X(q2)]
            )

In [3]:
def your_circuit(oracle):
    """Yields a circuit for the Deutsh-Joza algorithm on three qubits"""
    # phase kickback trick
    yield cirq.X(q2), cirq.H(q2)

    # equal supersposition over input bits
    yield cirq.H(q0), cirq.H(q1)

    # query the function
    yield oracle

    # interference to get result, put last qubit into |1>
    yield cirq.H(q0), cirq.H(q1), cirq.H(q2)

    # a final OR gate to put result in final qubit
    yield cirq.X(q0), cirq.X(q1), cirq.CCX(q0,q1,q2)
    yield cirq.measure(q2)

In [4]:
# get a simulator
sim = cirq.Simulator()

In [11]:
# Execute ciruit for oracles of constant value functions
print('Your result on constant funtions')
for oracle in constant:
    result = sim.run(cirq.Circuit(your_circuit(oracle)),repetitions=10)
print(result)

Your result on constant funtions
q(2)=0000000000


In [8]:
# Execute ciruit for oracles of balanced functions
print('Your result on balanced funtions')
for oracle in balanced:
    result = sim.run(cirq.Circuit(your_circuit(oracle)),repetitions=10)
print(result)

Your result on balanced funtions
q(2)=0000000000
